In [9]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV

In [3]:
train = pd.read_csv("train_set.csv")
train

,most_frequent__race_AfricanAmerican,most_frequent__race_Asian,most_frequent__race_Caucasian,most_frequent__race_Hispanic,most_frequent__race_Other,none__payer_code_BC,none__payer_code_CH,none__payer_code_CM,none__payer_code_CP,none__payer_code_DM,...,numerical_values__time_in_hospital,numerical_values__num_lab_procedures,numerical_values__num_procedures,numerical_values__num_medications,numerical_values__number_outpatient,numerical_values__number_emergency,numerical_values__number_inpatient,numerical_values__number_diagnoses,index,readmitted
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.799276,-1.267012,-0.783849,-0.860326,0.505522,1.927927,0.292447,0.826637,61710,>30
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.224536,0.567279,-0.783849,0.996918,0.505522,-0.210096,3.468418,0.312052,22317,NO
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.124672,1.586329,-0.783849,0.873101,0.505522,-0.210096,1.086440,0.826637,74460,NO
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.124672,1.229661,-0.783849,-0.736510,-0.290386,-0.210096,0.292447,-0.717119,9396,NO
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.124672,0.006801,-0.783849,-0.860326,-0.290386,-0.210096,-0.501545,0.312052,19435,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63572,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.124672,-0.197009,-0.783849,0.377837,0.505522,-0.210096,0.292447,-1.231704,25687,>30
63573,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.887234,0.465374,0.976785,-0.365061,-0.290386,-0.210096,-0.501545,-0.202533,45840,NO
63574,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.799276,-1.674632,-0.783849,-1.355591,-0.290386,-0.210096,-0.501545,0.826637,101244,NO
63575,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.124672,1.382519,-0.783849,0.006388,-0.290386,-0.210096,-0.501545,0.826637,42507,>30


In [4]:
test = pd.read_csv("test_set.csv")
test

,most_frequent__race_AfricanAmerican,most_frequent__race_Asian,most_frequent__race_Caucasian,most_frequent__race_Hispanic,most_frequent__race_Other,none__payer_code_BC,none__payer_code_CH,none__payer_code_CM,none__payer_code_CP,none__payer_code_DM,...,numerical_values__time_in_hospital,numerical_values__num_lab_procedures,numerical_values__num_procedures,numerical_values__num_medications,numerical_values__number_outpatient,numerical_values__number_emergency,numerical_values__number_inpatient,numerical_values__number_diagnoses,index,readmitted
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.212630,0.872994,-0.196971,0.501653,-0.290386,-0.210096,-0.501545,0.312052,1754,>30
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.461974,-1.317965,-0.196971,-0.117428,-0.290386,-0.210096,-0.501545,0.312052,15344,NO
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.461974,0.822041,0.976785,-0.117428,0.505522,0.858915,1.086440,0.312052,44597,NO
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.461974,0.006801,-0.196971,0.625469,0.505522,-0.210096,5.056404,-2.260874,18240,<30
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.124672,0.720136,-0.783849,0.254020,-0.290386,-0.210096,-0.501545,0.826637,66984,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19863,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.799276,-0.095104,2.150541,1.492183,-0.290386,-0.210096,-0.501545,0.826637,47396,<30
19864,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.573744,0.363469,-0.196971,0.006388,-0.290386,-0.210096,-0.501545,-0.202533,13348,>30
19865,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.236442,0.720136,-0.783849,-0.365061,-0.290386,-0.210096,0.292447,0.312052,97804,>30
19866,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.461974,0.159658,-0.783849,-0.860326,6.076883,-0.210096,-0.501545,0.826637,31011,>30


In [5]:
X_train = train.drop("readmitted", axis=1)
y_train = train.readmitted
X_test = test.drop("readmitted", axis=1)
y_test = test.readmitted

# Select the 25 best features using SelectKBest

In [8]:
selector = SelectKBest(k=25)
selector.fit(X_train, train.readmitted)
best_features = selector.get_feature_names_out()

# Function to find the best hyperparameters for k features

In [10]:
def get_best_hyperparams(X_train, y_train, param_grid, clf):
    cv = 5

    # Select K Best Features
    selector = SelectKBest(k=25)
    selector.fit(X_train, y_train)

    # Get the selected feature names
    best_features = selector.get_feature_names_out()

    # Use the best features for training
    train_features = X_train[best_features]
    print(train_features.shape)

    # Perform GridSearchCV to find the best parameters
    gridsearch = GridSearchCV(clf, param_grid, cv=cv, scoring='f1_macro', verbose=3)
    gridsearch.fit(train_features, y_train)
    
    # Print the best parameters and the best score
    print("Best params: ", gridsearch.best_params_, "Best score:", gridsearch.best_score_)

In [11]:
# build the base learners
log_clf = LogisticRegression(C=10.0, class_weight='balanced', penalty='l2')
gnb_clf = GaussianNB(var_smoothing=1e-09)
bernoulli_clf = BernoulliNB(alpha=0.1, binarize=0.0)
# build the ensemble
voting_clf = VotingClassifier(
estimators=[ ('lr', log_clf),
('bayes', gnb_clf),
('bernoulli', bernoulli_clf)]
)

voting_clf.fit(X_train[best_features], y_train)

# Evaluate the ensemble
ensemble_pred = voting_clf.predict(X_test[best_features])
ensemble_f1 = f1_score(y_test, ensemble_pred, average='macro')
print(classification_report(y_test, ensemble_pred))

              precision    recall  f1-score   support

         <30       0.26      0.16      0.20      2263
         >30       0.46      0.43      0.45      7100
          NO       0.63      0.70      0.66     10505

    accuracy                           0.55     19868
   macro avg       0.45      0.43      0.44     19868
weighted avg       0.53      0.55      0.53     19868

